In [1]:
def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return {tuple(list_of_partition[i]): row[i] for i in range(len(row)) if row[i] != 0}

def my_order(a):
    polynomial = to_polynomial(a)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(a), len(a) - number_of_ones(a), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def matrix_UVir_m2_m3(n):
    m2 = {(1, 1): 3, (2,): -4}
    m3 = {(1, 1, 1):4, (2, 1): -12, (3,): 3}
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 3)
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition2[i]), m2)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m3 = [L_partition_vector(tuple(list_of_partition3[i]), m3)
                         for i in range(len(list_of_partition3))]
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({(i, j): list_of_vector_m3[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m3(n)
    pivots = A_n.pivots()
    try:
        return to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

In [2]:
list_of_partition = [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1],
                     [6, 5, 3, 1]]
%time mW_list_of_partition(list_of_partition)

CPU times: user 285 ms, sys: 10.6 ms, total: 295 ms
Wall time: 295 ms


[{(2,): 1, (1, 1): -3/4},
 {(1, 1, 1): 1, (3,): -3},
 {(3, 1, 1): 1, (4, 1): -1, (5,): -3/2},
 {(3, 3): 1, (4, 1, 1): 1/3, (5, 1): -3/2, (6,): -2},
 {(4, 3, 1): 1,
  (4, 4): 1,
  (5, 3): 3/2,
  (6, 1, 1): -1,
  (7, 1): -9/2,
  (8,): -9/2},
 {(4, 4, 1): 1,
  (5, 3, 1): 9/8,
  (5, 4): 21/8,
  (6, 3): 3/4,
  (7, 1, 1): -19/8,
  (8, 1): -63/8,
  (9,): -27/4},
 {(5, 4, 1, 1): 1,
  (5, 5, 1): 21/8,
  (6, 4, 1): 9/2,
  (7, 3, 1): 9/8,
  (6, 5): 27/4,
  (7, 4): 27/8,
  (8, 3): -39/8,
  (9, 1, 1): -189/16,
  (10, 1): -117/4,
  (11,): -393/16},
 {(6, 5, 3, 1): 1,
  (6, 6, 3): 29/10,
  (7, 4, 4): -17/30,
  (7, 5, 3): 4,
  (8, 4, 3): 13/5,
  (7, 6, 1, 1): 231/20,
  (8, 5, 1, 1): 33/4,
  (9, 4, 1, 1): 81/20,
  (7, 7, 1): 2299/120,
  (8, 6, 1): 149/5,
  (9, 5, 1): 369/20,
  (10, 4, 1): -1/12,
  (11, 3, 1): -3681/160,
  (8, 7): 5807/160,
  (9, 6): 131/5,
  (10, 5): 567/80,
  (11, 4): -3973/160,
  (12, 3): -19159/160,
  (13, 1, 1): -140449/960,
  (14, 1): -12207/40,
  (15,): -4977/20}]